In [13]:
from pymongo import MongoClient
import requests
from SPARQLWrapper import SPARQLWrapper, JSON
from pprint import pprint
import csv

In [11]:
class MutantMongo(object):

    def __init__(self, mut_json=None):
        """

        :param mut_json:
        :return:
        """
        self.mut_json = mut_json
        self.client = MongoClient()
        self.genomes = self.client.genomes
        self.mutants = self.genomes.mutants
        self.pmids = {}
        self.insert_log = []

        self.mut_json['_id'] = '%s-%s-%s' % (self.mut_json['name'],
                                             self.mut_json['locusTag'],
                                             self.mut_json['coordinate']['start'])

    def push2mongo(self):
        try:
            self.mutants.insert_one(self.mut_json)
            return {
                    'write_success': True,
                    'duplicate_key': False,
                    }
        except Exception as e:
            print(e.args)
            if 'E11000' in e.args:
                return {
                    'write_success': False,
                    'duplicate_key': True,
                    }
            else:
                return {
                    'write_success': False,
                    'duplicate_key': False,
                    }

    def delete_one_mongo(self):
        try:
            self.mutants.delete_one({'_id': self.mut_json['_id']})
            return {'delete_success': True}

        except Exception as e:
            return {'delete_success': False}

    def add_gff_from_json(self):
        self.mut_json['gff'] = {
            "seqname": self.mut_json['chromosome'],
            "source": self.mut_json['mutant_type'],
            "feature": 'mutation',
            "start": self.mut_json['coordinate']['start'],
            "end": self.mut_json['coordinate']['start'],
            "score": '.',
            "strand": '.',
            "phase": '.',
            "attribute": 'id={}'.format(self.mut_json['name'])
        }

In [40]:
so_map = {
        'SYNONYMOUS' : {
            'name': 'synonymous',
            'id': 'SO:0001814'
        },
        'Non-neutral':{
            'name': 'non-synonymous',
            'id': 'SO:0001816'
        },
       'NON-CODING':{
            'name': 'non_transcribed_region',
            'id': 'SO:0000183'
        },
        'Neutral':{
            'name': 'silent_mutation',
            'id': 'SO:0001017'
        },
        'NONSENSE': {
            'name': 'stop_gained',
            'id': 'SO:0001587'
        }
    }
def get_pub(pmid):
    url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmed&retmode=json&id={}'.format(pmid)
    r = requests.get(url)
    return r.json()

with open('Kokes_mutants.csv', 'r') as km:
    count = 0
    for line in km:
        line = line.split(",")
        count += 1

        mut_json = {
            "taxid": "471472",
            "locusTag": line[1],
            "name": line[9],
            "chromosome": "NC_010287.1",
            "mutant_type": {
                "alias": "chemical mutagenesis",
                "name": "chemically induced mutation",
                "id": "EFO_0000370",
                "key": 1
            },
            "coordinate": {
                "start": line[0]
            },
              "ref_base": line[7],
              "variant_base": line[8],
            "variant_type": {
                "alias": line[5],
                "name": so_map[line[5]]["name"],
                "id": so_map[line[5]]["id"]
            },
            "aa_effect": line[6],
            "pub": get_pub(line[14]),

        }  
        MONGODB = MutantMongo(mut_json=mut_json)
        MONGODB.add_gff_from_json()
        MONGODB.push2mongo()


('E11000 duplicate key error collection: genomes.mutants index: _id_ dup key: { : "CTL2M_Pool 05-CTL0001-78" }',)
('E11000 duplicate key error collection: genomes.mutants index: _id_ dup key: { : "CTL2M_Pool 22-CTL0001-186" }',)
('E11000 duplicate key error collection: genomes.mutants index: _id_ dup key: { : "CTL2M_Pool 20-CTL0001-326" }',)
('E11000 duplicate key error collection: genomes.mutants index: _id_ dup key: { : "CTL2M_Pool 24-CTL0001-873" }',)
('E11000 duplicate key error collection: genomes.mutants index: _id_ dup key: { : "CTL2M_Pool 23-CTL0002-1180" }',)
('E11000 duplicate key error collection: genomes.mutants index: _id_ dup key: { : "CTL2M_Pool 36-CTL0002-1192" }',)
('E11000 duplicate key error collection: genomes.mutants index: _id_ dup key: { : "CTL2M_Pool 28-CTL0002-1351" }',)
('E11000 duplicate key error collection: genomes.mutants index: _id_ dup key: { : "CTL2M_Pool 34-CTL0002-1549" }',)
('E11000 duplicate key error collection: genomes.mutants index: _id_ dup key:

KeyError: ' ATPase component"'

In [ ]:
mut_json = {
  "taxid": "471472",
  "locusTag": "CTL0003",
  "name": "mut strain name",
  "chromosome": "NC_010287.1",
  "mutant_type": {
    "alias": "chemical mutagenesis",
    "name": "chemically induced mutation",
    "id": "EFO_0000370",
    "key": 1
  },
  "coordinate": {
    "start": "2001"
  },
  "ref_base": "C",
  "variant_base": "G",
  "variant_type": {
    "alias": "Non-neutral",
    "name": "non-synonymous",
    "id": "SO:0001816"
  },
  "aa_effect": "R127C",
  "pub": {
    "uid": "324324",
    "pubdate": "1977 May",
    "epubdate": "",
    "source": "Am Surg",
    "authors": [
      {
        "name": "Manberger AR",
        "authtype": "Author",
        "clusterid": ""
      }
    ],
    "lastauthor": "Manberger AR",
    "title": "\"To care for him who shall have borne the battle\"--Abraham Lincoln.",
    "sorttitle": "to care for him who shall have borne the battle abraham lincoln",
    "volume": "43",
    "issue": "5",
    "pages": "255-8",
    "lang": [
      "eng"
    ],
    "nlmuniqueid": "0370522",
    "issn": "0003-1348",
    "essn": "1555-9823",
    "pubtype": [
      "Historical Article",
      "Journal Article"
    ],
    "recordstatus": "PubMed - indexed for MEDLINE",
    "pubstatus": "4",
    "articleids": [
      {
        "idtype": "pubmed",
        "idtypen": 1,
        "value": "324324"
      },
      {
        "idtype": "rid",
        "idtypen": 8,
        "value": "324324"
      },
      {
        "idtype": "eid",
        "idtypen": 8,
        "value": "324324"
      }
    ],
    "history": [
      {
        "pubstatus": "pubmed",
        "date": "1977/05/01 00:00"
      },
      {
        "pubstatus": "medline",
        "date": "1977/05/01 00:01"
      },
      {
        "pubstatus": "entrez",
        "date": "1977/05/01 00:00"
      }
    ],
    "references": [],
    "attributes": [],
    "pmcrefcount": "",
    "fulljournalname": "The American surgeon",
    "elocationid": "",
    "doctype": "citation",
    "srccontriblist": [],
    "booktitle": "",
    "medium": "",
    "edition": "",
    "publisherlocation": "",
    "publishername": "",
    "srcdate": "",
    "reportnumber": "",
    "availablefromurl": "",
    "locationlabel": "",
    "doccontriblist": [],
    "docdate": "",
    "bookname": "",
    "chapter": "",
    "sortpubdate": "1977/05/01 00:00",
    "sortfirstauthor": "Manberger AR",
    "vernaculartitle": ""
  }
}

In [ ]:
client = MongoClient()
genomes = client.genomes
mutants = genomes.mutants
cursor = mutants.find()
counter = 0
for cur in cursor:
    counter += 1
    pprint(counter)

In [ ]:
cursor = mutants.find()
counter = 0
for cur in cursor:
    counter += 1
    pprint(counter)